## Importing libraries

In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## Loading dataset

In [6]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_valid, x_train = x_train_full[:5000], x_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [8]:
# Creating a dataset from the loaded data arrays
train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train))
valid_set = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [9]:
train_set

<ShuffleDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>

In [10]:
valid_set

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>